In [1]:
# First we load autoload, so we don't need to keep restarting the kernel to get
# new definitions. This ensures that functions are reloaded from the file whenever
# the file is changed.
%load_ext autoreload
%autoreload 2

In [ ]:
# All the stuff we need
from lark import *
from lark.tree import Tree
from lark.lexer import Token
from ast import literal_eval
import numpy as np

from helpers import *
from show import *
from type import *
from PE import * # Use your own partial evaluator here, or use reference implementation PEjames.py

from flatten  import *
from simulate import *
from zx_semantics import *

import pyzx
import pyzx.simplify as simplify

In [3]:
cqparse = Lark.open("CQ.lark",parser='lalr', start="program")
program = read_file("../CQ-programs/initialize.cq")
#program = read_file("../CQ-programs/qft2.cq")

program_tree=prune_tree(cqparse.parse(program, start="program"))

In [4]:
print(show_program(program_tree))


initialize_2qubit(float a[4],qbit q[2]) {
  float th1 = (2 * arccos(sqrt(((a[0] * a[0]) + (a[2] * a[2]))))) ;
  float th2 = (2 * arctan2(a[3],a[1])) ;
  float th3 = (2 * arctan2(a[2],a[0])) ;

  Ry(th1) q[1] ;
  Ry(th2) q[0] if q[1] ;
  not q[1] ;
  Ry(th3) q[0] if q[1] ;
  not q[1] ;
}



In [5]:
# The amplitudes 'a' are chosen such that ||a||^2 = 1/2 + 1/4 + 1/6 + 1/12 = 1
static_input = {'a': np.sqrt([1/2.,1/4.,1/6., 1/12.]),'d':4}

# Partially evaluate the CQ program with the classical input
qprog_tree = flatten_program(PE_program(program_tree, static_input))


In [6]:
# Let's look at the residual quantum program:
print(show_program(qprog_tree))

initialize_2qubit(qbit q[2]) {
  Ry(1.2309594173407747) q[1] ;
  Ry(1.0471975511965976) q[0] if q[1] ;
  not q[1] ;
  Ry(1.0471975511965976) q[0] if q[1] ;
  not q[1] ;
}



In [27]:
# Convert flat CQ- syntax tree to a PyZX circuit
zxc,qbits_env=zx_program(qprog_tree)

# Draw the circuit
print(list(zxc))
pyzx.draw(zxc)

# Convert PyZX circuit to free-form ZX-diagram
zxg = zxc.to_graph()
# Go look at https://github.com/zxcalc/pyzx/blob/master/pyzx/simplify.py#L300 and
# look at the what PyZX's full_reduce function does. 
#  You're going to go through individual steps by hand.
#pyzx.full_reduce(zxg)
pyzx.draw(zxg)

# Convert ZX-diagram to ZH-diagram (Z nodes with Hadamard edges)
# Most of the reduction steps work on this form.
pyzx.simplify.to_gh(zxg)
print(f"After to_gh");
pyzx.draw(zxg)



[QRot["exp(-i%Y)",0.6154797086703874](1), QRot["exp(-i%Y)",0.2617993877991494](0), CNOT(1,0), QRot["exp(-i%Y)",-0.2617993877991494](0), CNOT(1,0), NOT(1), QRot["exp(-i%Y)",0.2617993877991494](0), CNOT(1,0), QRot["exp(-i%Y)",-0.2617993877991494](0), CNOT(1,0), NOT(1)]


After to_gh


In [30]:
# Run through the ZX-rewrite rules involved in full_reduce one by one
# Repeat until no more rules apply (all removed_xxx are 0).
# In each reduction, follow along on paper as best you can and understand what 
# rule has been used on which part of the diagram. 
# Refer to https://arxiv.org/pdf/2012.13966.pdf Appendix A for the rules.
# Note that blue edges are Hadamard edges (instead of drawing with a square).
print("After to_gh");
pyzx.simplify.to_gh(zxg)
pyzx.draw(zxg)

removed_id = simplify.id_simp(zxg,quiet=False)
if(removed_id>0): 
    zxg.normalize(); print(f"After {removed_id} Id-simplification (Section 4.1)."); pyzx.draw(zxg)

removed_spider = simplify.spider_simp(zxg,quiet=False)
if(removed_spider>0): 
    zxg.normalize(); print(f"After {removed_spider} spider-simplifications. (Section 4.1)"); pyzx.draw(zxg)

removed_pivot  = simplify.pivot_simp(zxg,quiet=False)
if(removed_pivot>0):
    zxg.normalize(); print(f"After {removed_pivot} Pivot-simplification (Section 6.4)."); pyzx.draw(zxg)

removed_lcomp = simplify.lcomp_simp(zxg,quiet=False)
if(removed_lcomp>0):
    zxg.normalize(); print(f"After {removed_lcomp} Lcomp-simplification (Section 6.4)."); pyzx.draw(zxg)

removed_bialg  = simplify.bialg_simp(zxg,quiet=False)
if(removed_bialg>0):
    zxg.normalize(); print(f"After {removed_bialg} Bialg-simplification (Section 4.5)."); pyzx.draw(zxg)

removed_gadget = simplify.gadget_simp(zxg,quiet=False) + simplify.pivot_gadget_simp(zxg,quiet=False)
if(removed_gadget>0):
    zxg.normalize(); print(f"After {removed_gadget} Gadget-simplification (Section 5.6)."); pyzx.draw(zxg)

print(f"After to_rg");
pyzx.simplify.to_rg(zxg)
pyzx.draw(zxg)
print(removed_id, removed_spider, removed_pivot,removed_bialg, removed_gadget, removed_lcomp)

After to_gh


id_simp: 1.  1 iterations
After 1 Id-simplification (Section 4.1).


spider_simp: 1.  1 iterations
After 1 spider-simplifications. (Section 4.1)


lcomp_simp: 1.  1 iterations
After 1 Lcomp-simplification (Section 6.4).


After to_rg


1 1 0 0 0 1


In [31]:

pyzx.draw(zxg)
# Extract a circuit from the ZX-diagram
zxc2 = pyzx.extract_simple(zxg.copy())
pyzx.draw(zxc2)

print(f"Old circuit: {zxc.stats()}\nNew circuit: {zxc2.stats()}")


Old circuit: Circuit  on 2 qubits with 11 gates.
        5 is the T-count
        6 Cliffords among which
        4 2-qubit gates (4 CNOT, 0 other) and
        0 Hadamard gates.
There are 5 gates of a different type
New circuit: Circuit  on 2 qubits with 8 gates.
        2 is the T-count
        6 Cliffords among which
        0 2-qubit gates (0 CNOT, 0 other) and
        2 Hadamard gates.


In [32]:
# Now let's turn this back into a CQ- program
new_stats = list(statements_from_zxc(zxc2,qbits_env))

for s in new_stats:
    print(show_statement(s))

H q[1] ;
H q[0] ;
SX q[1] ;
SX q[0] ;
Rz(0.33983690945412187) q[1] ;
Rz(0.5235987755982989) q[0] ;
SX q[1] ;
SX q[0] ;


In [33]:
# rewrite_Qprogram_statements produces a new CQ- program with same input and declarations, 
# but with the qupdate statements replaced.
qprog2=rewrite_Qprogram_statements(qprog_tree,new_stats)

In [34]:
print(show_program(qprog_tree))
print(show_program(qprog2))

initialize_2qubit(qbit q[2]) {
  Ry(1.2309594173407747) q[1] ;
  Ry(1.0471975511965976) q[0] if q[1] ;
  not q[1] ;
  Ry(1.0471975511965976) q[0] if q[1] ;
  not q[1] ;
}

initialize_2qubit(qbit q[2]) {
  H q[1] ;
  H q[0] ;
  SX q[1] ;
  SX q[0] ;
  Rz(0.33983690945412187) q[1] ;
  Rz(0.5235987755982989) q[0] ;
  SX q[1] ;
  SX q[0] ;
}



In [255]:
# Now we check that the transformation is semantics preserving.
# Calculate the matrix-representation of the original and transformed program:
Mp=simulate_program(qprog_tree).reshape(4,4)
Mp2=simulate_program(qprog2).reshape(4,4)

#print(np.round(Mp,2))
#print(abs(Mp @ np.array([1,0,0,0]))** -2)

In [ ]:
# Run the original program on the |00> input state (represented by (1,0,0,0))
Mvec = Mp @ np.array([1,0,0,0])
abs(Mvec)** -2

array([ 2.,  4.,  6., 12.])

In [240]:
# Run the transformed program on the |00> input state (represented by (1,0,0,0))
M2vec = Mp2 @ np.array([1,0,0,0])
abs( M2vec )** -2

array([ 2.,  4.,  6., 12.])